## Analyse textuelle du débat sur l'Article 9

In [1]:
#coding: utf-8
#importing modules
#lecture des json
import json
#text-minig
import re
#viz
import matplotlib
import matplotlib.pyplot as plt
#calcul
import numpy as np
from itertools import combinations
%matplotlib inline

## Article 9: version originale

In [2]:
def load_vo(art_file = "./article_9.json"):
    with open(art_file, "r") as f:
        article9 = json.load(f)
    return article9
article9 = load_vo()


* **découvrir les informations disponibles dans l'article9**

In [3]:
# toutes les clés du dictionnaire de l'article9
print("Clés de l'ARTICLE 9:")
for clé, valeur in list(article9.items()):
    print("-"+clé, type(valeur))

Clés de l'ARTICLE 9:
-author <class 'str'>
-ranking <class 'NoneType'>
-versions_count <class 'int'>
-created_at <class 'str'>
-sources <class 'list'>
-versions <class 'list'>
-article_link <class 'str'>
-body <class 'str'>
-updated_at <class 'str'>
-title <class 'str'>
-answer <class 'dict'>
-article_id <class 'int'>
-votes_ok <class 'int'>
-arguments <class 'list'>
-sources_count <class 'int'>
-body_anchors <class 'list'>
-votes_nok <class 'int'>
-cat_id <class 'int'>
-votes_mitige <class 'int'>
-arguments_count <class 'int'>
-body_links <class 'list'>
-votes_total <class 'int'>
-subtitle <class 'str'>


L'article9 stocke à la fois les informations contextuelles sur l'article dans sa version originale mais encore
tous les **arguments**, les **sources** et toutes les **versions** de cet article.
On note que les **votes** sont pour le moment dans un fichier à part.
Nous allons garder de coté les arguments, sources et versions pour les manipuler 

In [4]:
#on stocke les informations dans des variables simples
versions = article9["versions"] #toutes les versions sans la version originale
arguments = article9["arguments"] #tous les arguments (version orginale+ versions complémentaires??? a verifier)
sources = article9["sources"] #toutes les sources


### Versions de l'article 9

Ici nous allons nous concentrer sur les différentes versions (amendements) de l'article 
ainsi que les **commentaires** qui justifient ces modifications


* **découvrir les informations contenues dans les versions**:

*pour l'exemple nous allons prendre la première version de ses versions complémentaires (soit V1)*


In [5]:
v1 = versions[0]
print("Les informations pour une version")
for clé, valeur in v1.items():
    print("-"+clé, type(valeur))

Les informations pour une version
-author <class 'str'>
-before <class 'str'>
-id <class 'int'>
-link <class 'str'>
-after <class 'str'>
-created_at <class 'str'>
-arguments_count <class 'int'>
-votes_nok <class 'int'>
-arguments_no_count <class 'int'>
-votes_mitige <class 'int'>
-votes_total <class 'int'>
-updated_at <class 'str'>
-arguments_yes_count <class 'int'>
-slug <class 'str'>
-comment <class 'str'>
-title <class 'str'>
-article_id <class 'int'>


Les valeurs qui vont nous intéresser dans un premier temps vont etre:
* le titre
* le comment

* ** combien de versions? **


In [6]:
print(len(versions), "versions complémentaires + 1 version originale")  

108 versions complémentaires + 1 version originale


### Formatter la version originale
Nous allons dans un premier temps formatter la version originale de la même manière que les versions complémentaire.
avec les mêmes informations disponibles dans chaque version
La version originale avec les informations qui nous intéressent sous forme de dictionnaire (de la même manière que les autres versions)

En sachant que :
* la version originale n'a pas de comment mais une explication de l'article (body) ainsi qu'une réponse aux multiples arguments (answer)
* le titre complet n'est pas stocké dans l'article on se servira dans un premier temps du subtitle
* les votes mitigés ne sont pas comptabilisés
* la repartition des arguments entre ok et pas ok n'est pas disponible (il faudrait mapper chaque argument de cette version pour en avoir le coeur net (plus tard)



In [7]:
v0 = {"link": article9["article_link"], 
      "title": article9["subtitle"], 
      "after": article9["body"], 
      "comment": article9["answer"], 
      "votes_ok": article9["votes_ok"],
      "votes_nok": article9["votes_nok"],
      "votes_mitige": article9["votes_total"] - (article9["votes_ok"]+article9["votes_nok"]),
      "votes_total": article9["votes_total"], 
      "created_at": article9["created_at"],
      "udpated_at": article9["updated_at"],
      "author": "gouvernement",
      "id": article9["article_id"],
      "arguments_count": article9["arguments_count"],
      #manque 
      #arguments_yes_count
      #arguments_no_count
     }

### Rappatrier la version originale dans les versions

Nous allons donc rappatrier **v0** (notre version originale) dans notre variable **versions** qui est une *liste*.
La variable version est une liste ordonnée qui respecte l'ordre chronologique des version:
nous allons donc insérer en position 0

In [8]:
versions.insert(0, v0)


In [9]:
print(versions[0])

{'author': 'gouvernement', 'id': 69, 'link': 'http://www.republique-numerique.fr/projects/projet-de-loi-numerique/consultation/consultation/opinions/section-2-travaux-de-recherche-et-de-statistique/article-9-acces-aux-travaux-de-la-recherche-financee-par-des-fonds-publics', 'after': 'Au chapitre 3 du titre 3 du livre V du code de la recherche, il est inséré un article L. 533-4 ainsi rédigé\xa0:«Art. L. 533-4 –I. Lorsque un écrit scientifique, issu d’une activité de recherche financée au moins pour moitié par des fonds publics, est publié dans un périodique, un ouvrage paraissant au moins une fois par an, des actes de congrès ou de colloques ou des recueils de mélanges, son auteur, même en cas de cession exclusive à un éditeur, dispose du droit de mettre à disposition gratuitement sous une forme numérique, sous réserve des droits des éventuels coauteurs, la dernière version acceptée de son manuscrit par son éditeur et à l’exclusion du travail de mise en forme qui incombe à ce dernier, a

In [10]:
print(versions[1])

{'author': 'philippegambette', 'before': '', 'id': 181, 'link': 'http://www.republique-numerique.fr/projects/projet-de-loi-numerique/consultation/consultation/opinions/section-2-travaux-de-recherche-et-de-statistique/article-9-acces-aux-travaux-de-la-recherche-financee-par-des-fonds-publics/versions/mise-a-disposition-systematique-des-resultats-de-la-recherche-financee-par-des-fonds-publics', 'after': 'I. Lorsque un écrit scientifique, issu d’une activité de recherche financée au moins pour moitié par des fonds publics est publié dans un périodique, un ouvrage paraissant au moins une fois par an, des actes de congrès ou de colloques ou des recueils de mélanges,son auteur, même en cas de cession exclusive à un éditeur, dispose dudroit de mettre à disposition gratuitement sous une forme numérique, sous réserve des droits des éventuels co-auteurs, la dernière version acceptée de son manuscrit par son éditeur et à l’exclusion du travail de mise en forme qui incombe à ce dernier, au terme d

* ** Combien de versions?**:

In [11]:
print(len(versions),"versions")

109 versions


* **Combien de votes et d'arguments par version?**

In [27]:
print ("version_nb\t|votes nb\t|arguments nb\t|auteur")
print("---------------------------------------------------------------------")
for i,v in enumerate(versions):
    print ("version_"+str(i)+"\t|", v["votes_total"], "\t\t|", v["arguments_count"], "\t\t|", v["author"])

version_nb	|votes nb	|arguments nb	|auteur
---------------------------------------------------------------------
version_0	| 3334 		| 132 		| gouvernement
version_1	| 66 		| 9 		| philippegambette
version_2	| 0 		| 0 		| melaniedulongderosnay
version_3	| 3 		| 1 		| alainbensoussan
version_4	| 1 		| 0 		| melaniedulongderosnay
version_5	| 1 		| 0 		| lefebvrenarefrederic
version_6	| 0 		| 0 		| tanataricoalessandra
version_7	| 0 		| 0 		| watine
version_8	| 6 		| 0 		| adbsassociationdesprofessionnelsdelinformationetdeladocumentation1
version_9	| 0 		| 0 		| sbauin
version_10	| 2 		| 0 		| nicolaslarchet
version_11	| 78 		| 2 		| epristassociationdesresponsablesistdesorganismesderecherchefrancais
version_12	| 3 		| 2 		| jeannevarasco
version_13	| 7 		| 0 		| julianbh
version_14	| 17 		| 0 		| durandaline
version_15	| 4 		| 2 		| dominiquelecomte
version_16	| 11 		| 2 		| alainlecavelier
version_17	| 8 		| 1 		| hervemorel
version_18	| 71 		| 3 		| federationnationaledelapressedinforma

Ecrivons ces données dans un fichier pour plus tard (on ne sait jamais)
ici une version minimale:

In [28]:

with open("liste_des_versions.csv", "w") as f:
    f.write("version_nb\tvotes nb\targuments nb\tauteur\n")
    for i,v in enumerate(versions):
        f.write("version_"+str(i)+"\t"+str(v["votes_total"])+"\t"+str(v["arguments_count"])+"\t"+v["author"]+"\n")
#NB: vu qu'on manipule des listes on pourrait l'écrire grace à la fonction join() qui va avec les listes
#Si on voulait faire de la visualisation le Notebook 
Cartographie%20des%20comportements%20des%20electeurs.ipynb


#### Les commentaires des versions
L'objectif ici est de faire un état des lieux des mots employés pour justifier les modifications apportés à la version originale 
dans une version complémentaire:

Dans un premier temps, il s'agit de vérifier l'état des modifications:
* Y-a-t-il eu beaucoup de modification du texte original? 
* Combien de mots d'une version à l'autre ?
* Combien de mots au total de la version initiale à la version finale?

Dans un deuxième temps, il s'agira de plonger dans le détail et regarder les mots communs et différents à chaque versions
* Combien de mots en commun de version a version+1
* Combien de mots différents de version a version+1
* Combien de mots communs/différents au total de la version initiale à la version finale?


Dans un troisième temps, il faudra plonger dans le détail des auteurs et des électeurs
en effet un auteur a parfois fait plusieurs modifications.
* Combien d'auteur unique?
* Combien de versions par auteurs?
* Chaque auteur a-t-il un vocabulaire particulier? 
* Analyser la proximité/distance lexicale des auteurs entre eux (mots communs/mots différents)



Dans un quatrième temps, il faudra rappatrier tous les electeurs qui ont voté pour la modification de versions
* Quels sont les auteur les plus populaires en nombre de votes absolus?
* Quels sont les auteurs les plus proches (en regardant les electeurs qui ont voté pour la même personne)

